In [1]:
from models.reinforcement_learning import *

In [2]:
track_file = 'datasets/R-track.txt'
racetrack = Racetrack(track_file)

In [3]:
# Assuming your Simulator class and environment are correctly set up
simulator = Simulator(racetrack, 'nearest')  # Pass necessary parameters like racetrack and options

# Create a ValueIteration instance with the simulator
value_iteration = ValueIteration(simulator, gamma=0.99, theta=0.1)

# Run the Value Iteration algorithm
value_iteration.run()

# Retrieve the computed optimal policy
optimal_policy = value_iteration.get_policy()


In [4]:
# Print the optimal policy to see the recommended action for each state
for state, action in optimal_policy.items():
    print(f"State: {state}, Action: {action}")

State: (1, 12, -5, -5), Action: (-1, -1)
State: (1, 12, -5, -4), Action: (-1, -1)
State: (1, 12, -5, -3), Action: (-1, -1)
State: (1, 12, -5, -2), Action: (-1, -1)
State: (1, 12, -5, -1), Action: (-1, -1)
State: (1, 12, -5, 0), Action: (-1, -1)
State: (1, 12, -5, 1), Action: (-1, -1)
State: (1, 12, -5, 2), Action: (-1, -1)
State: (1, 12, -5, 3), Action: (-1, -1)
State: (1, 12, -5, 4), Action: (-1, -1)
State: (1, 12, -5, 5), Action: (-1, -1)
State: (1, 12, -4, -5), Action: (-1, -1)
State: (1, 12, -4, -4), Action: (-1, -1)
State: (1, 12, -4, -3), Action: (-1, -1)
State: (1, 12, -4, -2), Action: (-1, -1)
State: (1, 12, -4, -1), Action: (-1, -1)
State: (1, 12, -4, 0), Action: (1, -1)
State: (1, 12, -4, 1), Action: (-1, -1)
State: (1, 12, -4, 2), Action: (-1, -1)
State: (1, 12, -4, 3), Action: (-1, -1)
State: (1, 12, -4, 4), Action: (-1, -1)
State: (1, 12, -4, 5), Action: (-1, -1)
State: (1, 12, -3, -5), Action: (-1, -1)
State: (1, 12, -3, -4), Action: (-1, -1)
State: (1, 12, -3, -3), Actio

In [5]:
initial_state = (simulator.start_x, simulator.start_y, 0, 0)  # Example initial state
steps_taken = simulator.simulate_with_policy(optimal_policy)
print(f"Total steps taken to reach the finish: {steps_taken}")


Step 0: State (3, 26, 0, 0) -> Action (-1, -1) -> New State (2, 25, -1, -1), Reward -1
Step 1: State (2, 25, -1, -1) -> Action (1, -1) -> New State (2, 23, 0, -2), Reward -1
Step 2: State (2, 23, 0, -2) -> Action (0, -1) -> New State (2, 20, 0, -3), Reward -1
Step 3: State (2, 20, 0, -3) -> Action (0, -1) -> New State (2, 16, 0, -4), Reward -1
Step 4: State (2, 16, 0, -4) -> Action (0, -1) -> New State (2, 11, 0, -5), Reward -1
Step 5: State (2, 11, 0, -5) -> Action (1, 1) -> New State (2, 6, 0, -5), Reward -1
Step 6: State (2, 6, 0, -5) -> Action (-1, -1) -> New State (2, 5, 0, 0), Reward -1
Step 7: State (2, 5, 0, 0) -> Action (1, -1) -> New State (3, 4, 1, -1), Reward -1
Step 8: State (3, 4, 1, -1) -> Action (1, 0) -> New State (5, 3, 2, -1), Reward -1
Step 9: State (5, 3, 2, -1) -> Action (1, 0) -> New State (8, 2, 3, -1), Reward -1
Step 10: State (8, 2, 3, -1) -> Action (1, 0) -> New State (12, 1, 4, -1), Reward -1
Step 11: State (12, 1, 4, -1) -> Action (1, 1) -> New State (17, 1

In [6]:
print(racetrack.track[12][3])

.
